## 13.3 時間と空間を表現するグラフ

In [ ]:
import pandas as pd
import json
from plotly import express as px
from plotly import graph_objects as go
from plotly.graph_objs.layout import Template

# 都道府県別新規陽性者数のDataFrameを読み込み
df = pd.read_csv('newly_confirmed_cases_daily.csv', index_col=0, parse_dates=True)

df.info()

In [ ]:
df.index

In [ ]:
# 都道府県のGeoJsonを読み込み
with open('todofuken.geojson') as file:
    geojson = json.load(file)

In [ ]:
# 都道府県のローマ字と漢字のdictを作成
todofuken_dict = {}
for feature, todofuken in zip(geojson['features'], list(df.columns)[1:]):
    todofuken_dict[todofuken] = feature['properties']['N03_001']

todofuken_dict

In [ ]:
# DataFrameの列名を漢字表記に変更
df = df.rename(columns=todofuken_dict)
df = df.iloc[:, 1:]     # 先頭列の「ALL」は除外する

df.info()

In [ ]:
# 新規陽性者数を年間合計
df_year = df.resample('Y').sum()

df_year.shape

In [ ]:
# メルトするために行インデックスを列として追加
df_year['Date'] = df_year.index

# 年間新規陽性者数を縦に並べるようDataFrameをメルト
df_melt = df_year.melt(id_vars=['Date'], var_name='都道府県', value_name='年間新規陽性者数')
df_melt['Date'] = df_melt['Date'].astype(str)

df_melt

In [ ]:
# 年でグループ化
gb = df_melt.groupby('Date')

# Frameのlistを作成
frames = []
max_value = df_melt['年間新規陽性者数'].max()
for group, df_subset in gb:
    trace = go.Choropleth(
        locations=df_subset['都道府県'],
        z=df_subset['年間新規陽性者数'],
        geojson=geojson,
        featureidkey='properties.N03_001',
        colorscale='Reds',
        zmin=0,
        zmax=max_value
    )   # コロプレスマップ
    frame = go.Frame(
        data=trace,
        name=group, 
        layout={
            'title': {'text': f'新型コロナウイルス年間新規陽性者数 {group}'}
        }
    )
    frames.append(frame)

In [ ]:
# ステップを作成
steps = []
for date in list(gb.groups.keys()):
    steps.append({
        'args': [
            [date],
        ],
        'label': date,
        'method': 'animate'
    })

# スライダーを作成
sliders = [{
    'len': 0.95,    # スライダー長さ
    'x': 0.05,      # スライダー左位置
    'steps': steps
}]

# 再生ボタン
play_button = {
    'args': [
        None,
        {'fromcurrent': True}    # 現在位置から再生再開する
    ],
    'label': 'Play',
    'method': 'animate'
}

# 一時停止ボタン
pause_button = {
    'args': [
        [None],
        {'mode': 'immediate'}   # 停止するために必要
    ],
    'label': 'Pause',
    'method': 'animate'
}

# ボタンメニューを作成
button_menu = {
    'buttons': [play_button, pause_button],
    'direction': 'left',    # 2つのボタンを並べる方向
    'xanchor': 'left',      # xアンカー位置
    'yanchor': 'top',       # yアンカー位置
    'x': -0.1,              # x位置
    'y': -0.15,             # y位置
    'type': 'buttons'
}

In [ ]:
# 独自テンプレートを読み込む
with open('custom_white.json') as f:
    custom_white_dict = json.load(f)
    template = Template(custom_white_dict, _validate=False)

# Layoutを作成
layout = go.Layout(
    template=template,
    margin={
        'r': 20,
        't': 30,
        'l': 20,
        'b': 30
    },
    geo={
        'fitbounds': 'locations',
        'visible': False,
    },
    updatemenus=[button_menu],
    sliders=sliders,  
)

# Figureを作成
figure = go.Figure(
    data=frames[0]['data'],     # 最初に表示するグラフ
    frames=frames,
    layout=layout
)

figure